I. Environment setup:
- Spark 3.1.2 with Hadoop 3.2.
- Java 8.
- Mount driver.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


II. Code: 

II.1. Quickstart:




II.1.1. Basics: 

In [ ]:
!pip install pyspark

In [22]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [31]:
textFile = spark.read.text("/content/drive/MyDrive/HUNT/Speech Recognition/README.md")

In [27]:
textFile.count()

1

In [28]:
textFile.first()

Row(value='# Speech Recognition:')

In [42]:
linesWithSpark = textFile.filter(textFile.value.contains("Spark"))

In [32]:
textFile.filter(textFile.value.contains("Spark")).count()  # How many lines contain "Spark"?

1

II.1.1. More on Dataset Operations:

In [35]:
from pyspark.sql.functions import *

In [36]:
textFile.select(size(split(textFile.value, "\s+")).name("numWords")).agg(max(col("numWords"))).collect()

[Row(max(numWords)=4)]

In [37]:
wordCounts = textFile.select(explode(split(textFile.value, "\s+")).alias("word")).groupBy("word").count()

In [38]:
wordCounts.collect()

[Row(word='Colab.', count=1),
 Row(word='Speech', count=1),
 Row(word='Technologies:', count=1),
 Row(word='Hadoop.', count=1),
 Row(word='I.', count=1),
 Row(word='-', count=4),
 Row(word='Google', count=1),
 Row(word='#', count=1),
 Row(word='##', count=1),
 Row(word='Spark', count=1),
 Row(word='Python.', count=1),
 Row(word='Recognition:', count=1)]

II.1.1. Cache: 



In [43]:
linesWithSpark.cache()

DataFrame[value: string]

In [44]:
linesWithSpark.count()

1

In [45]:
linesWithSpark.count()

1